# Import Packages:

In [224]:
import pandas as pd
import networkx as nx
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import requests
import regex
import re
import pyvis as pv
from IPython.display import display,HTML

# The Data:

### I have chosen a network of Marvel Superheros for my network. Each node represents a character or a comic book/ movie that they can be linked to. The dataset can be found at: http://konect.cc/networks/marvel/. The network is bipartite, where edges link a super hero to a comic that they were present in.

In [225]:
## Read in node information:
hero_nodes = pd.read_csv("https://raw.githubusercontent.com/sphill12/Data_620/main/heros.txt", sep = "/n", header = None)
hero_nodes = pd.DataFrame(data = {"hero":hero_nodes[0].apply(lambda x: str(x).strip())})
hero_nodes.head()

C:\Users\Steve\AppData\Local\Temp\ipykernel_35396\250878319.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  hero_nodes = pd.read_csv("https://raw.githubusercontent.com/sphill12/Data_620/main/heros.txt", sep = "/n", header = None)


,hero
0,Vertex 1: 24-HOUR MAN/EMMANUEL
1,Vertex 2: 3-D MAN/CHARLES CHANDLER & HAROLD CH...
2,Vertex 3: 4-D MAN/MERCURIO
3,Vertex 4: 8-BALL/
4,Vertex 5: A


### After reading in the hero names, a node can be assigned to them, and a dictionary can be created to store the metadata on the node:

In [226]:
## Format the nodes so that they can be read into a networkx graph

## Use a regex to remove the vertex information before hero names
hero_nodes = pd.DataFrame(hero_nodes["hero"].apply(lambda x: re.sub("V[^:]*: ","",x)))

## Add node list
hero_nodes["node"] = [i for i in range(1,6487)]


## Create a dictionary that can be read into graph object
hero_nodes["meta_data"] = hero_nodes["hero"].apply(lambda x: {"hero" : x})
hero_nodes["node_data"] = list(zip(hero_nodes.node, hero_nodes.meta_data))
hero_nodes

,hero,node,meta_data,node_data
0,24-HOUR MAN/EMMANUEL,1,{'hero': '24-HOUR MAN/EMMANUEL'},"(1, {'hero': '24-HOUR MAN/EMMANUEL'})"
1,3-D MAN/CHARLES CHANDLER & HAROLD CHANDLER,2,{'hero': '3-D MAN/CHARLES CHANDLER & HAROLD CH...,"(2, {'hero': '3-D MAN/CHARLES CHANDLER & HAROL..."
2,4-D MAN/MERCURIO,3,{'hero': '4-D MAN/MERCURIO'},"(3, {'hero': '4-D MAN/MERCURIO'})"
3,8-BALL/,4,{'hero': '8-BALL/'},"(4, {'hero': '8-BALL/'})"
4,A,5,{'hero': 'A'},"(5, {'hero': 'A'})"
...,...,...,...,...
6481,FORTUNE,6482,{'hero': 'FORTUNE'},"(6482, {'hero': 'FORTUNE'})"
6482,SEERESS,6483,{'hero': 'SEERESS'},"(6483, {'hero': 'SEERESS'})"
6483,STORMER,6484,{'hero': 'STORMER'},"(6484, {'hero': 'STORMER'})"
6484,TIGER WYLDE,6485,{'hero': 'TIGER WYLDE'},"(6485, {'hero': 'TIGER WYLDE'})"


### Now Read in the dataframe that contains the comics that superheros were in:


In [227]:
## The comic nodes start at 6487 
comics_nodes = pd.read_csv("https://raw.githubusercontent.com/sphill12/Data_620/main/comics.txt", sep ="/n", header = None)

comics_nodes = pd.DataFrame(data = {"comic": comics_nodes[0].apply(lambda x : re.sub("V[^:]*: ", "", x))})

C:\Users\Steve\AppData\Local\Temp\ipykernel_35396\2634176876.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  comics_nodes = pd.read_csv("https://raw.githubusercontent.com/sphill12/Data_620/main/comics.txt", sep ="/n", header = None)


### Now generate the metadata dictionary for the comic nodes:

In [228]:
## Generate the list of nodes, at 6487 where they start
index = [i for i in range(6487, (6487 + 12942), 1)]
comics_nodes["node"] = index

## Create a dictionary so that nodes and metadata can be read into network object
comics_nodes["metadata"] = comics_nodes["comic"].apply(lambda x: {"comic":x})
comics_nodes["node_data"] = list(zip(comics_nodes.node,comics_nodes.metadata))
comics_nodes.head()

,comic,node,metadata,node_data
0,AA2 35,6487,{'comic': 'AA2 35'},"(6487, {'comic': 'AA2 35'})"
1,M/PRM 35,6488,{'comic': 'M/PRM 35'},"(6488, {'comic': 'M/PRM 35'})"
2,M/PRM 36,6489,{'comic': 'M/PRM 36'},"(6489, {'comic': 'M/PRM 36'})"
3,M/PRM 37,6490,{'comic': 'M/PRM 37'},"(6490, {'comic': 'M/PRM 37'})"
4,WI? 9,6491,{'comic': 'WI? 9'},"(6491, {'comic': 'WI? 9'})"


### The edgelist is then read in:

In [229]:
## read the edges in

edge_list = pd.read_csv("https://raw.githubusercontent.com/sphill12/Data_620/main/node_list.txt", delim_whitespace= True)
edge_list = edge_list.iloc[:,0:2]
edge_list = edge_list.rename(columns = {"%":"node_1","bip":"node_2"})
edge_list["edges"] = list(zip(edge_list.node_1,edge_list.node_2))
edge_list.head()

C:\Users\Steve\AppData\Local\Temp\ipykernel_35396\1841840711.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  edge_list = pd.read_csv("https://raw.githubusercontent.com/sphill12/Data_620/main/node_list.txt", delim_whitespace= True)


,node_1,node_2,edges
0,1,6487,"(1, 6487)"
1,2,6488,"(2, 6488)"
2,2,6489,"(2, 6489)"
3,2,6490,"(2, 6490)"
4,2,6491,"(2, 6491)"


### Now the bipartite graph is ready to be generated:

In [230]:
## Create a bipartite graph object

B = nx.Graph()
B.add_nodes_from(hero_nodes["node_data"], bipartite = 0, key  = int)
B.add_nodes_from(comics_nodes["node_data"],bipartite = 1, key = int)
B.add_edges_from(edge_list["edges"])

### To simplify the analysis, it is better to project the graph, as we only care about the relationships between the hero nodes:

In [231]:
## Create a projected graph 

G = nx.bipartite.weighted_projected_graph(B, hero_nodes["node"])

### We will be using the island method to look at the connections between nodes that have a high weight. Prior to trimming edges, we can look at the size of our network:

In [232]:
print("There are " +  str(len(list(G.nodes))) + " nodes")
print("There are " +  str(len(list(G.edges))) + " edges")

There are 6486 nodes
There are 168267 edges


### Now that we have our projected graph, we can start by trimming low weight edges to get an idea of how the network is stuctured:

In [233]:
## trim the edges of the network

def trim_edges(g, weight = 1):
    g2 = nx.Graph()
    for f, to, edata in g.edges(data = True):
        if edata["weight"] > weight:
            g2.add_edge(f,to, weight = edata["weight"])
    return g2

trimmed_network = trim_edges(G)

In [234]:
print("There are " +  str(len(list(trimmed_network.nodes))) + " nodes")
print("There are " +  str(len(list(trimmed_network.edges))) + " edges")

There are 4568 nodes
There are 77227 edges


### Just by trimming the network of nodes with a weight of 1, we have removed about 2000 nodes and about half of the edges that existed before

### We can look at centrality in the network before we begin to form islands:

In [235]:
temp = hero_nodes[["hero", "node"]]
trimmed_centrality = pd.DataFrame(list(nx.degree_centrality(G).items()),columns = ["node","degree_centrality"])
trimmed_centrality = pd.merge(temp, trimmed_centrality, on = "node")
trimmed_centrality.sort_values(by = "degree_centrality", ascending=False).head(25)

,hero,node,degree_centrality
858,CAPTAIN AMERICA,859,0.298072
5305,SPIDER-MAN/PETER PARKER,5306,0.268466
2663,IRON MAN/TONY STARK,2664,0.235621
5715,THING/BENJAMIN J. GRIMM,5716,0.219892
6305,WOLVERINE/LOGAN,6306,0.214958
3804,MR. FANTASTIC/REED RICHARDS,3805,0.213724
2556,HUMAN TORCH/JOHNNY STORM,2557,0.211411
4897,SCARLET WITCH/WANDA MAXIMOFF,4898,0.207402
5735,THOR/DR. DONALD BLAKE/SIGURD JARLSON II/JAKE O...,5736,0.198766
402,BEAST/HENRY &HANK& P. MCCOY,403,0.197379


### Unsurprisingly, the heros with the highest centrality are those that are some of Marvel's biggest superheros. Captain America, Spiderman, Iron Man, and Wolvarine. In Superhero comics, there are generally "communities" of heros that are formed. For example, Captain America and Wolverine don't interact with eachother often as wolverine is part of the X-men community and Captain America is part of the avengers community. We expect these heroes to potentially form their own islands

In [236]:
## show subgraphs based on the amount of connections
def island_method(g, iterations = 10):
    weights = [edata["weight"] for f,to, edata in g.edges(data = True)]
    
    mn = int(min(weights))
    mx = int(max(weights))
    step = int((mx-mn)/ iterations)
    return [[threshold, trim_edges(g, threshold)] for threshold in range(mn, mx, step)]

### Use the island method to find interesting islands:

In [237]:
islands = island_method(G, iterations = 20)
print("repeats per node, nodes, islands that are available")
for i in islands:
    print(i[0], len(i[1]), len(list(nx.connected_components(i[1]))))

repeats per node, nodes, islands that are available
1 4568 12
38 319 17
75 168 10
112 98 8
149 67 6
186 50 7
223 41 7
260 32 5
297 27 5
334 23 6
371 19 5
408 11 4
445 9 3
482 7 2
519 7 2
556 6 2
593 6 2
630 4 1
667 4 1
704 3 1
741 2 1


### We can see that using edge weights between 112 and 261 will most likely net us the most interesting islands. Anything less could be too cluttered, and anything more will be removing too many nodes. I will start by visualizing a smaller network so that we can see the most important nodes first.

# Create and Visualize Island Networks:

### First a function to create dataframes from the trimmed data is generated. This build_community function takes the network (or list of subnetworks) that was trimmed using the island method, and generates graphs for each community. It then calculates the centrality of each node. For later visualization purposes, it assigns a color to each node based on the level of centrality. The node with the highest overall centrality will be red, those over the 75th percentile of centrality will be orange, above 50% will be yellow, above 25% will be blue, and the rest will be grey.

In [238]:
## This function can be used on the communities generated by the previous function to create a dataframe which can then be used to visualize the network

def build_community(community_list, input_type = "node",):
    ## Use community node or edge or subgraph list to create subgraph from main network
    if input_type == "node":
        network = trimmed_network.subgraph(community_list)
    if input_type == "edge":
        network = trimmed_network.edge_subgraph(community_list)
    if input_type == "connected_components":
        index = 0
        group = {}
        network = nx.Graph()
        connected_graphs = [community_list.subgraph(c).copy() for c in sorted(nx.connected_components(community_list), key = len, reverse =True)]
        for graph in connected_graphs:
            index += 1
            network.add_edges_from(list(graph.edges()))
            for node in list(graph.nodes()):
                group[node] = index
            
    ## Initiate dataframe with super hero nodes that are in subgraph
    data_frame = hero_nodes[hero_nodes["node"].isin(list(network.nodes))]
    ## Calculate the centrality of the nodes in the subgraph 
    centrality = pd.DataFrame(data = list(nx.degree_centrality(network).items()), columns = ["node","deg_centrality"])
    data_frame = data_frame[["hero","node"]]
    
    ## merge node and centrality information
    data_frame= pd.merge(data_frame, centrality, on = "node")
    if input_type == "connected_components":
        between_centrality = pd.DataFrame(data = list(nx.betweenness_centrality(network).items()),columns = ["node", "between_centrality"])
        group = pd.DataFrame(group.items(), columns = ["node", "group"])
        data_frame = pd.merge(data_frame, group, on = "node")
        data_frame = pd.merge(data_frame, between_centrality, on = "node")
    ## Assign colors based on centrality percentiles
    percentiles = data_frame["deg_centrality"].describe()
    temp = []
    
    ## loop through nodes to assign color
    for row in data_frame["deg_centrality"]:
        ## highest centrality node is red
        if row == percentiles["max"]:
            temp.append("red")
        elif row > percentiles["75%"]:
            temp.append("orange")
        elif row > percentiles["50%"]:
            temp.append("yellow")
        elif row > percentiles["25%"]:
            temp.append("blue")
        else:
            temp.append("grey")
    
    ## Add color list to dataframe
    data_frame["colors"] = temp
    return data_frame

### Next, a function to visualize this networks easily is created:

In [239]:
## Create a function to build networks plots:

def build_visual(data_frame, specific_network, physics = True):
    ## initiate network from dataframe and subgraph
    
    network = specific_network.subgraph(list(data_frame["node"]))
    ## set visual properties
    net = pv.network.Network(height = "1000px", width = "100%", notebook = False)
    
    ## Set physics parameters
    net.barnes_hut(spring_strength = 0.006)
    
    ## Add nodes in pyviz format
    pyviz_nodes = list(data_frame["node"])
    net.add_nodes(pyviz_nodes, title = list(data_frame["hero"]), color= list(data_frame["colors"]), label = list(data_frame["hero"]))
    
    ## Add edges in pyviz format
    pyviz_edges = list(network.edges)
    pyviz_edges = [list(ele) for ele in pyviz_edges]
    net.add_edges(pyviz_edges)
    
    ## Show plot of network 
    net.show("network2.html", notebook = False)

### We will start by visualizing the highest weight threshold to see the smallest islands:

In [253]:
viz_graph = trim_edges(G, 261)

In [254]:
viz_df = build_community(viz_graph, input_type="connected_components")
build_visual(viz_df, viz_graph)
viz_df.sort_values(by = "between_centrality", ascending= False).head(10)

network2.html


,hero,node,deg_centrality,group,between_centrality,colors
5,CYCLOPS/SCOTT SUMMERS,1289,0.214286,1,0.076720,orange
28,WOLVERINE/LOGAN,6306,0.142857,1,0.066138,yellow
14,NIGHTCRAWLER/KURT WAGNER,3950,0.107143,1,0.023810,blue
24,THOR/DR. DONALD BLAKE/SIGURD JARLSON II/JAKE O...,5736,0.107143,2,0.018519,blue
3,CAPTAIN AMERICA,859,0.250000,2,0.016975,red
10,IRON MAN/TONY STARK,2664,0.250000,2,0.016975,red
21,SPIDER-MAN/PETER PARKER,5306,0.142857,3,0.013228,yellow
4,COLOSSUS II/PETER RASPUTIN,1127,0.107143,1,0.002646,blue
19,SCARLET WITCH/WANDA MAXIMOFF,4898,0.178571,2,0.001764,orange
6,HAWK,2399,0.178571,2,0.000661,orange


In [242]:
viz_df.sort_values(by = "deg_centrality", ascending = False).head(10)

,hero,node,deg_centrality,group,between_centrality,colors
10,IRON MAN/TONY STARK,2664,0.250000,2,0.016975,red
3,CAPTAIN AMERICA,859,0.250000,2,0.016975,red
5,CYCLOPS/SCOTT SUMMERS,1289,0.214286,1,0.076720,orange
26,WASP/JANET VAN DYNE PYM,6148,0.178571,2,0.000661,orange
6,HAWK,2399,0.178571,2,0.000661,orange
19,SCARLET WITCH/WANDA MAXIMOFF,4898,0.178571,2,0.001764,orange
0,ANGEL/WARREN KENNETH WORTHINGTON III,133,0.142857,1,0.000000,yellow
21,SPIDER-MAN/PETER PARKER,5306,0.142857,3,0.013228,yellow
1,ANT-MAN/DR. HENRY J. PYM,154,0.142857,2,0.000000,yellow
12,MARVEL GIRL/JEAN GREY SUMMERS,3495,0.142857,1,0.000000,yellow


## Now we can look at a lower weight cutoff for more connected islands:

In [255]:
viz_graph = trim_edges(G,112)
viz_df = build_community(viz_graph, input_type = "connected_components")
build_visual(viz_df, viz_graph)


network2.html


In [256]:
viz_df.sort_values(by ="between_centrality",ascending =False).head(10)

,hero,node,deg_centrality,group,between_centrality,colors
12,CAPTAIN AMERICA,859,0.268041,1,0.243254,red
4,BEAST/HENRY &HANK& P. MCCOY,403,0.123711,1,0.197201,orange
80,SPIDER-MAN/PETER PARKER,5306,0.134021,1,0.123711,orange
87,THOR/DR. DONALD BLAKE/SIGURD JARLSON II/JAKE O...,5736,0.216495,1,0.103588,orange
42,IRON MAN/TONY STARK,2664,0.175258,1,0.073570,orange
81,STORM/ORORO MUNROE Subscribe!,5467,0.144330,1,0.049568,orange
94,WOLVERINE/LOGAN,6306,0.134021,1,0.048179,orange
8,BLACK WIDOW/NATASHA ROMANOVA,545,0.051546,1,0.043868,yellow
88,VISION,6066,0.144330,1,0.030035,orange
85,THING/BENJAMIN J. GRIMM,5716,0.113402,1,0.029968,orange


In [245]:
viz_df.sort_values(by=  "deg_centrality",ascending  = False).head(10)

,hero,node,deg_centrality,group,between_centrality,colors
12,CAPTAIN AMERICA,859,0.268041,1,0.243254,red
87,THOR/DR. DONALD BLAKE/SIGURD JARLSON II/JAKE O...,5736,0.216495,1,0.103588,orange
42,IRON MAN/TONY STARK,2664,0.175258,1,0.073570,orange
88,VISION,6066,0.144330,1,0.030035,orange
81,STORM/ORORO MUNROE Subscribe!,5467,0.144330,1,0.049568,orange
90,WASP/JANET VAN DYNE PYM,6148,0.134021,1,0.018520,orange
80,SPIDER-MAN/PETER PARKER,5306,0.134021,1,0.123711,orange
94,WOLVERINE/LOGAN,6306,0.134021,1,0.048179,orange
4,BEAST/HENRY &HANK& P. MCCOY,403,0.123711,1,0.197201,orange
85,THING/BENJAMIN J. GRIMM,5716,0.113402,1,0.029968,orange


In [257]:
viz_graph = trim_edges(G, 38)
viz_df = build_community(viz_graph, input_type = "connected_components")
build_visual(viz_df, viz_graph)

network2.html


# Using networkX algorithms to locate islands:

### Networkx also has algorithms available for generating islands/communities. If we want to trim less off the network and visualize more connected individual islands, then the community functions can be useful:

In [258]:
communities_trim = trim_edges(G, 5)
communities = sorted(nx.community.louvain_communities(communities_trim), key = len, reverse =True)

for i in communities:
    print(i)

{6146, 2051, 1029, 4104, 3080, 1038, 1039, 4118, 6168, 5146, 6173, 3101, 6175, 2081, 2083, 5156, 37, 39, 1065, 2091, 5164, 5165, 5173, 3128, 57, 2106, 3134, 1087, 5190, 3143, 6216, 76, 3149, 2126, 1115, 6235, 2142, 5214, 4197, 102, 1127, 4198, 6249, 107, 3181, 1138, 3188, 6266, 126, 4223, 1155, 133, 6279, 3209, 6283, 1165, 2191, 4239, 6288, 6290, 5117, 4240, 1175, 4247, 1177, 6300, 3231, 5280, 6306, 1186, 1188, 1189, 3237, 6311, 6312, 168, 172, 1198, 4272, 3250, 1204, 1206, 183, 186, 6330, 189, 2238, 2237, 1214, 4289, 195, 5317, 1223, 204, 5327, 6351, 2256, 2258, 4306, 5337, 6366, 6367, 5345, 5347, 3307, 4338, 251, 6398, 3326, 1280, 3329, 4354, 1282, 1289, 5386, 2316, 4365, 4366, 1299, 3347, 4375, 280, 1304, 4378, 6427, 284, 1309, 6430, 3359, 3360, 3357, 288, 3363, 2345, 4395, 4397, 3374, 2358, 3383, 3397, 4427, 1356, 3406, 1361, 340, 2390, 5466, 5467, 2397, 3425, 3426, 1381, 1382, 1383, 5484, 5485, 1392, 1397, 5497, 5498, 5500, 5502, 1407, 5504, 3459, 281, 3470, 403, 5524, 3477, 3476,

In [259]:
communities_df = build_community(communities[0], input_type="node")
build_visual(communities_df, communities_trim)

network2.html


## We can look at the size of each of these communities that have been generated:

In [260]:
communities_df.sort_values(by = "deg_centrality", ascending = False).head(10)

,hero,node,deg_centrality,colors
383,WOLVERINE/LOGAN,6306,0.665816,red
322,STORM/ORORO MUNROE Subscribe!,5467,0.625000,orange
81,CYCLOPS/SCOTT SUMMERS,1289,0.619898,orange
65,COLOSSUS II/PETER RASPUTIN,1127,0.586735,orange
258,PROFESSOR X/CHARLES FRANCIS XAVIER,4366,0.584184,orange
21,BEAST/HENRY &HANK& P. MCCOY,403,0.576531,orange
206,MARVEL GIRL/JEAN GREY SUMMERS,3495,0.563776,orange
7,ANGEL/WARREN KENNETH WORTHINGTON III,133,0.561224,orange
235,NIGHTCRAWLER/KURT WAGNER,3950,0.545918,orange
298,SHADOWCAT/KATHERINE KITTY PRYDE,5002,0.505102,orange
